## Name: Saif Ur Rahman
## Roll Number: 2022-10-0001

#knowledge Distillation 
In this assignment we will impliment [TCN](https://www.bmvc2021-virtualconference.com/conference/papers/paper_0831.html) paper. It is a varient of knowledge distillation which uses dense feature vactors instead of logits to transfer knowledge from teacher to student.  

# Training base teacher network
This section is not graded

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim
import os
from torch.autograd import Variable
import tqdm

batch_size = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=100, shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

cuda


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

teacher = VGG('VGG16')
teacher = teacher.to(device)

In [3]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(teacher.parameters(), lr=0.0001)

def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    teacher.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = Variable(inputs, requires_grad=False)
        targets = Variable(targets)
        teacher.zero_grad()
        outputs = teacher(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if(batch_idx % 200 == 0):
          print("Accuracy : ",100.*correct/total," Loss : ", train_loss/(batch_idx+1))
def test(epoch):
    teacher.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = teacher(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            if(batch_idx % 20 == 0):
              print("Accuracy : ",100.*correct/total," Loss : ", test_loss/(batch_idx+1))

In [4]:
start_epoch = 0
best_acc = 0
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    print("Validation: ")
    test(epoch)


Epoch: 1
Accuracy :  8.0  Loss :  2.449977397918701
Accuracy :  41.82089552238806  Loss :  1.5786146643149912
Accuracy :  48.96758104738154  Loss :  1.401194090706452
Validation: 
Accuracy :  69.0  Loss :  0.938983678817749
Accuracy :  64.76190476190476  Loss :  1.0078013056800479
Accuracy :  64.51219512195122  Loss :  1.003554972206674
Accuracy :  64.29508196721312  Loss :  1.0004605142796625
Accuracy :  64.14814814814815  Loss :  1.0051931863949624

Epoch: 2
Accuracy :  66.0  Loss :  1.010141372680664
Accuracy :  64.33333333333333  Loss :  0.9950507594578302
Accuracy :  66.07231920199501  Loss :  0.9523365898917143
Validation: 
Accuracy :  77.0  Loss :  0.7040292620658875
Accuracy :  71.33333333333333  Loss :  0.8100343119530451
Accuracy :  71.41463414634147  Loss :  0.805798401192921
Accuracy :  72.04918032786885  Loss :  0.7971230403321689
Accuracy :  72.07407407407408  Loss :  0.8013275854381514

Epoch: 3
Accuracy :  72.0  Loss :  0.7292823195457458
Accuracy :  72.43781094527363 

# Creating Dense Feature Dataset
1.1 In this cell you need to remove the head of teacher network(i.e: last fullyconnected layer) and add a flatten layer at the end.

In [5]:
teacher_WOH = nn.Sequential(*(list(teacher.children())[:-1]), nn.Flatten())
from torchsummary import summary
summary(teacher_WOH, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]          73,856
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12

The summery of your new teacher without head should look something like this

In [ ]:
from torchsummary import summary
summary(teacher_WOH, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]          73,856
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12

1.2 In this cell you have to create dense feature labels dataset(i.e: the outputs of teacher network without head). For that you have to do forward pass on whole dataset and append the outputs in a variable. 

In [6]:
teacher_WOH.eval()
DenseTrain = {}
DenseTest = {}
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs = inputs.to(device)
        #code here
        outputs = teacher_WOH(inputs)
        DenseTrain[batch_idx] = outputs
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs = inputs.to(device)
        #code here
        outputs = teacher_WOH(inputs)
        DenseTest[batch_idx] = outputs

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# def load_dict(filepath):
#   with open(file_path, 'r') as file:
#       data = json.load(file)

#Creating ad-hoc student network
1.3 Now you need to create an ad-hoc student network that must have less than 2M parameters. for that, you can add a CFG in the following code to design your student network. The output shape(i.e fully connected layer) should match the shape of dense features so that Mean Squared Error can be applied between them.

In [ ]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'VGGS': [64, 'M', 128, 'M', 128, 'M', 256, 'M', 512, 'M'],

}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 512)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

s1 = VGG('VGGS')
s1 = s1.to(device)
summary(s1, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]         147,584
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 256, 4, 4]         295,168
      BatchNorm2d-14            [-1, 25

# Training Student
1.4 Here you will train the student network using Dense Features that you created in 1.2 as targets. Dataset datagen will provide data in batches so you need to extract the corresponding batch of targets from your Dense feature variable from 1.2, for this you can use the following formula:

batch_index * batch_size --> (batch_index * batch_size) + batch_size

In [ ]:
optimizer = optim.Adam(s1.parameters(), lr=0.0001)
criterion = nn.MSELoss()

def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    s1.train()
    train_loss = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        #code here
        batch_size=100
        targets = DenseTrain[batch_idx]
        targets.to(device)
        targets = Variable(targets)
        s1.zero_grad()
        
        outputs = s1(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if(batch_idx % 10 == 0):
          print("Loss : ", train_loss/(batch_idx+1))
def test(epoch):
    s1.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            #code here
            batch_size=100
            targets = DenseTest[batch_idx]
            targets.to(device)
            targets = Variable(targets)
            s1.zero_grad()
            
            outputs = s1(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            if(batch_idx % 20 == 0):
              print(" Loss : ", test_loss/(batch_idx+1))

In [ ]:
start_epoch = 0
best_acc = 0
for epoch in range(start_epoch, start_epoch+100):
    train(epoch)
    print("Validation: ")
    test(epoch)

Streaming output truncated to the last 5000 lines.
Loss :  0.09991147411383199
Loss :  0.0999928440923437
Loss :  0.10012034052041503
Loss :  0.10014550903547088
Loss :  0.1001330200977732
Loss :  0.09999764778949093
Validation: 
 Loss :  0.07969352602958679
 Loss :  0.0883280641975857
 Loss :  0.08759218690598883
 Loss :  0.08810015914381528
 Loss :  0.08800661343115347

Epoch: 15
Loss :  0.1078830435872078
Loss :  0.10051297735084187
Loss :  0.09872171921389443
Loss :  0.0975584342114387
Loss :  0.09764004171621508
Loss :  0.09823545871996413
Loss :  0.09787525151108133
Loss :  0.0980628091894405
Loss :  0.09755374114086598
Loss :  0.09759268508507656
Loss :  0.09734466111305917
Loss :  0.09747878717141109
Loss :  0.09724893488667229
Loss :  0.09713373373027977
Loss :  0.09735549679884674
Loss :  0.09741633661732768
Loss :  0.09738623304581791
Loss :  0.09741798500742829
Loss :  0.09742673745800777
Loss :  0.09728172417554556
Loss :  0.09716998307562585
Loss :  0.09726009101240556
Lo

In [ ]:
# save model parameters
PATH = "/content/gdrive/MyDrive/Deep Learning/A5/s1_model"
torch.save(s1.state_dict(), PATH)

#Finetuning student on crossentropy
1.5 Now the student is trained. In this cell you need to replace the classifier(i.e: Fully Connected layer) of student network from one with output shape of dense feature to one with shape of classes. e.g: nn.Linear(256,512) to nn.Linear(256,10). After this you need to freez Conv layers in the network and finetune the network using orignal dataset. 

In [ ]:
s1.classifier = nn.Linear(512, 10)
for i, m in enumerate(s1.modules()):
  print(i, '->', m)
  print(m.parameters())

0 -> VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, 

In [ ]:
s1.classifier = nn.Linear(512, 10)

# This is found by printing the model summary abouve using the s1.module() and then finding the index of convolutional layers
index_of_conv_layers = [2,6,10,14,18]

for i, m in enumerate(s1.modules()):
    # only update if the index matches index of a conv2d layer
    if i in index_of_conv_layers:
      for param in m.parameters():
        param.requires_grad = False
s1 = s1.to(device)
summary(s1, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]         147,584
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 256, 4, 4]         295,168
      BatchNorm2d-14            [-1, 25

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(s1.parameters(), lr=0.0001)

def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    s1.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = Variable(inputs, requires_grad=False)
        targets = Variable(targets)
        s1.zero_grad()
        outputs = s1(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if(batch_idx % 200 == 0):
          print("Accuracy : ",100.*correct/total," Loss : ", train_loss/(batch_idx+1))
def test(epoch):
    s1.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = s1(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            if(batch_idx % 20 == 0):
              print("Accuracy : ",100.*correct/total," Loss : ", test_loss/(batch_idx+1))

In [ ]:
start_epoch = 0
best_acc = 0
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    print("Validation: ")
    test(epoch)


Epoch: 1
Accuracy :  3.0  Loss :  2.4733028411865234
Accuracy :  54.76119402985075  Loss :  1.647194907439882
Accuracy :  66.94513715710723  Loss :  1.3137869756120697
Validation: 
Accuracy :  76.0  Loss :  0.7670699954032898
Accuracy :  81.38095238095238  Loss :  0.7331053870064872
Accuracy :  80.7560975609756  Loss :  0.7469167898340923
Accuracy :  80.80327868852459  Loss :  0.7457917498760536
Accuracy :  80.66666666666667  Loss :  0.7424066868829139

Epoch: 2
Accuracy :  84.0  Loss :  0.7549332976341248
Accuracy :  82.78109452736318  Loss :  0.6767922316617634
Accuracy :  83.3790523690773  Loss :  0.632064999413312
Validation: 
Accuracy :  79.0  Loss :  0.5862411260604858
Accuracy :  82.71428571428571  Loss :  0.5528255601723989
Accuracy :  82.34146341463415  Loss :  0.5667841986912053
Accuracy :  82.39344262295081  Loss :  0.5643514595070823
Accuracy :  82.33333333333333  Loss :  0.5607238746719596

Epoch: 3
Accuracy :  89.0  Loss :  0.4995458722114563
Accuracy :  84.4278606965174

**Finetuned student model** has greater final validation accuracy of **85.0%** as compared to the teacher model which had final accuracy of 83.0%

#Multi Students
The paper impliments multiple students but till now you have implimented only one student. In this part you will create two smaller students, train them on different chunks of Dense features(i.e: DenseTrain[:,:256] for one student and DenseTrain[:,256:] for other student) and then ensamble them in the final output network. Constraint is that the sum of both students should not be greater then single student in terms of perimeters. 

#Creating Multiple students
1.6 In this step you will create two students smaller in size with outputs 256 each. 

In [9]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    'VGGS':  [64, 'M', 64, 'M', 128, 'M', 128, 'M', 256, 'M'],

}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(256, 256)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

s1 = VGG('VGGS')
s1 = s1.to(device)
summary(s1, (3, 32, 32))
s2 = VGG('VGGS')
s2 = s2.to(device)
summary(s2, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,928
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 128, 4, 4]         147,584
      BatchNorm2d-14            [-1, 12

Sum of parameters of both student models is: 622,400*2 = 1,244,800. This is less than 2M parameters

#Training Multi Students
1.6 In this step you will train two students instead of one. In the training loop you will pass the input from both students and then backwark both the losses. 

In [27]:
optimizer1 = optim.Adam(s1.parameters(), lr=0.0001)
optimizer2 = optim.Adam(s2.parameters(), lr=0.0001)
criterion = nn.MSELoss()

def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    s1.train()
    train_loss1 = 0
    train_loss2 = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        #code here
        # load the targets which are dense feature vectors
        batch_size=100
        targets = DenseTrain[batch_idx]
        targets.to(device)
        targets = Variable(targets)
        
        s1.zero_grad()
        s2.zero_grad()
        
        # find outputs of both models
        outputs1 = s1(inputs)
        outputs2 = s2(inputs)

        # find loss by splitting dense vector of teacher into 2 subspaces so
        # that each student learns a separate subspace
        targets1 = 0
        targets2 = 0
        for i in range(targets.shape[0]):
            if i == 0:
              targets1 = torch.unsqueeze(targets[i][:256], dim=0)
              # print("why is this happening", targets1.shape)
              targets2 = torch.unsqueeze(targets[i][256:], dim=0)
            else:
              targets1 = torch.cat((targets1, torch.unsqueeze(targets[i][:256], dim=0)),0)
              targets2 = torch.cat((targets2, torch.unsqueeze(targets[i][256:], dim=0)),0)

        # print("targets1 shape:", targets1.shape)
        # print("targets2 shape:", targets2.shape)
        # find the loss according to the criteria
        loss1 = criterion(outputs1, targets1)
        loss2 = criterion(outputs2, targets2)
        
        # backpropagation steps to update weights
        loss1.backward()
        loss2.backward()
        
        optimizer1.step()
        optimizer2.step()

        train_loss1 += loss1.item()
        train_loss2 += loss2.item()
        if(batch_idx % 10 == 0):
          print("Loss S1: ", train_loss1/(batch_idx+1))
          print("Loss S2: ", train_loss2/(batch_idx+1))
def test(epoch):
    s1.eval()
    test_loss1 = 0
    test_loss2 = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            #code here
            # load the targets which are dense feature vectors
            batch_size=100
            targets = DenseTrain[batch_idx]
            targets.to(device)
            targets = Variable(targets)
            
            s1.zero_grad()
            s2.zero_grad()
            
            # find outputs of both models
            outputs1 = s1(inputs)
            outputs2 = s2(inputs)

            # find loss by splitting dense vector of teacher into 2 subspaces so
            # that each student learns a separate subspace
            targets1 = 0
            targets2 = 0
            for i in range(targets.shape[0]):
                # print("what")
                if i == 0:
                  targets1 = torch.unsqueeze(targets[i][:256], dim=0)
                  # print("why is this happening", targets1.shape)
                  targets2 = torch.unsqueeze(targets[i][256:], dim=0)
                else:
                  targets1 = torch.cat((targets1, torch.unsqueeze(targets[i][:256], dim=0)),0)
                  targets2 = torch.cat((targets2, torch.unsqueeze(targets[i][256:], dim=0)),0)

            # print("targets shape:", targets2.shape)

            loss1 = criterion(outputs1, targets1)
            loss2 = criterion(outputs2, targets2)
            
            test_loss1 += loss1.item()
            test_loss2 += loss2.item()
            if(batch_idx % 20 == 0):
              print(" Loss S1: ", test_loss1/(batch_idx+1))
              print(" Loss S2: ", test_loss2/(batch_idx+1))

In [28]:
start_epoch = 0
best_acc = 0
for epoch in range(start_epoch, start_epoch+100):
    train(epoch)
    print("Validation: ")
    test(epoch)

Streaming output truncated to the last 5000 lines.
Loss S2:  0.0721916065997741
Loss S1:  0.0730183014995695
Loss S2:  0.07215476217469523
Loss S1:  0.07304882083580831
Loss S2:  0.07215465575007164
Loss S1:  0.07298981008920269
Loss S2:  0.07205922154882868
Loss S1:  0.07308571368555007
Loss S2:  0.07213517478326471
Loss S1:  0.07307825045995495
Loss S2:  0.07213230309866514
Loss S1:  0.0730904453768219
Loss S2:  0.07217536214936818
Loss S1:  0.07308103602666122
Loss S2:  0.07217740264417229
Loss S1:  0.07298367190537045
Loss S2:  0.07209080476655251
Validation: 
 Loss S1:  1.6104755401611328
 Loss S2:  1.5243847370147705
 Loss S1:  1.5639704011735462
 Loss S2:  1.5196441468738375
 Loss S1:  1.555942401653383
 Loss S2:  1.5103547311410672
 Loss S1:  1.5490869361846173
 Loss S2:  1.5014789593024331
 Loss S1:  1.5524690401406935
 Loss S2:  1.5024688567644284

Epoch: 57
Loss S1:  0.07264567911624908
Loss S2:  0.06728608161211014
Loss S1:  0.07374458014965057
Loss S2:  0.07197785038839687

In [29]:
# save model parameters
PATH = "/content/gdrive/MyDrive/Deep Learning/A5/s1_multistudent_model"
torch.save(s1.state_dict(), PATH)
PATH = "/content/gdrive/MyDrive/Deep Learning/A5/s2_multistudent_model"
torch.save(s2.state_dict(), PATH)

#Create ensamble model

1.8 In this step you will create a new network class that takes s1, and s2 as perimeters. This class should initiate a new network that ensembles both s1 and s2, and have a classifier for cross-entropy. In the forward method pass the input x from both s1 and s2 and then concatenate there outputs along axis 1. Then pass this concatinated output through classifier of appropriate shape. 

In [34]:
class Net(nn.Module):
    def __init__(self, s1,s2):
        super(Net, self).__init__()
        self.s1 = s1
        self.s2 = s2
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        #Code here
        # print(x.shape)
        x1 = self.s1(x)
        x2 = self.s2(x)
        # print("x1 shape", x1.shape)
        # print("x2 shape", x2.shape)
        x = torch.cat((x1, x2),1)
        # print(x.shape)
        out = self.classifier(x)
        # print(out.shape)
        return out
net = Net(s1,s2)
net.to(device)
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,928
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 128, 4, 4]         147,584
      BatchNorm2d-14            [-1, 12

#Train The Ensambled network
1.9 In this step you will freez all the conv layers in the ensambled network and then finetune it on orignal dataset. 

In [35]:
for i, m in enumerate(net.modules()):
  print(i, '->', m)
  print(m.parameters())



0 -> Net(
  (s1): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): Ba

In [36]:
# This is found by printing the model summary abouve using the s1.module() and then finding the index of convolutional layers
index_of_conv_layers = [3,7,11,15,19,27,31,35,39,43]

for i, m in enumerate(net.modules()):
    # only update if the index matches index of a conv2d layer
    if i in index_of_conv_layers:
      for param in m.parameters():
        param.requires_grad = False

net = net.to(device)
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,928
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 128, 4, 4]         147,584
      BatchNorm2d-14            [-1, 12

Take note that the trainable parameters are much less than the actual number of parameters. This indicates the the conv2d layers were switched off.

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

def train(epoch):
    print('\nEpoch: %d' % (epoch+1))
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = Variable(inputs, requires_grad=False)
        targets = Variable(targets)
        net.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if(batch_idx % 200 == 0):
          print("Accuracy : ",100.*correct/total," Loss : ", train_loss/(batch_idx+1))
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            if(batch_idx % 20 == 0):
              print("Accuracy : ",100.*correct/total," Loss : ", test_loss/(batch_idx+1))

In [38]:
start_epoch = 0
best_acc = 0
for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    print("Validation: ")
    test(epoch)


Epoch: 1
Accuracy :  7.0  Loss :  2.6424481868743896
Accuracy :  72.8407960199005  Loss :  0.9775601062015514
Accuracy :  78.95511221945137  Loss :  0.7290290362222533
Validation: 
Accuracy :  84.0  Loss :  0.5044693946838379
Accuracy :  83.71428571428571  Loss :  0.47616819398743765
Accuracy :  83.60975609756098  Loss :  0.48366556516507775
Accuracy :  83.75409836065573  Loss :  0.4782984203002492
Accuracy :  83.39506172839506  Loss :  0.47503023015128243

Epoch: 2
Accuracy :  88.0  Loss :  0.39672794938087463
Accuracy :  85.29353233830845  Loss :  0.4263456021227054
Accuracy :  85.57356608478803  Loss :  0.4203893430438125
Validation: 
Accuracy :  85.0  Loss :  0.4743480384349823
Accuracy :  84.19047619047619  Loss :  0.45898051985672544
Accuracy :  84.1219512195122  Loss :  0.4659295391018798
Accuracy :  84.26229508196721  Loss :  0.46032412594459093
Accuracy :  84.04938271604938  Loss :  0.45722021088923936

Epoch: 3
Accuracy :  89.0  Loss :  0.3189612329006195
Accuracy :  85.8805

**Finetuned student model** with **2 children** has even better final validation accuracy of **88.0%**. This final validation accuracy is greater than that achieved by a single student finetuned model (85%) and teacher model (83%). This suggests that the student networks were able to learn a more accurate representation of the underlying model depicted by the training set.

#4 and 8 Students (Bonus)
2.0 In this step try to divide dense feature into four and eight chunks and try to create respective smaller students. 